![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_RoBertaForSequenceClassification.ipynb)

# Import OpenVINO RoBertaForSequenceClassification models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting RoBertaForSequenceClassification models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for RoBertaForSequenceClassification from RoBertaForSequenceClassification   and they have to be in `Text Classification` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.41.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.35.1
!pip install -q --upgrade openvino==2024.3
!pip install -q --upgrade optimum-intel==1.18.3
!pip install -q --upgrade onnx==1.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.34.2 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.17.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 18.6 MB/s eta 0:00:0

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [deepset/roberta-base-squad2](https://huggingface.co/deepset/roberta-base-squad2) model from HuggingFace as an example and load it as a `OVModelForQuestionAnswering`, representing an OpenVINO model.
- In addition to the OVModelForQuestionAnswering model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from optimum.intel import OVModelForSequenceClassification
from transformers import AutoTokenizer

MODEL_NAME = "textattack/roberta-base-imdb"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Save the OpenVINO model
ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

# Create directory for assets and move the tokenizer files.
# A separate folder is needed for Spark NLP.
!mkdir {EXPORT_PATH}/assets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

Framework not specified. Using pt to export the model.


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at textattack/roberta-base-imdb were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Using framework PyTorch: 2.4.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> False
Compiling the model to CPU ...


In [ ]:
import json

# Read the vocab JSON file
with open('{}/vocab.json'.format(EXPORT_PATH), 'r') as json_file:
    tokenizer = json.load(json_file)

# let's save the vocab as txt file
with open('{}/vocab.txt'.format(EXPORT_PATH), 'w') as keys_file:
  for item in tokenizer.keys():
    keys_file.write("%s\n" % item)

In [ ]:
# get label2id dictionary
labels = ov_model.config.id2label
# sort the dictionary based on the id
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [ ]:
!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets
!mv {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets

## Import and Save RoBertaForSequenceClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2023-10-16 21:08:22--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-10-16 21:08:23--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-10-16 21:08:23 (93.8 MB/s) - written to stdout [1191/1191]

Installing

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `RoBertaForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

sequenceClassifier = RoBertaForSequenceClassification.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your RoBertaForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 487524
drwxr-xr-x 5 root root      4096 Oct 16 21:15 fields
drwxr-xr-x 2 root root      4096 Oct 16 21:15 metadata
-rw-r--r-- 1 root root 499209257 Oct 16 21:16 roberta_classification_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaForSequenceClassification model 😊

In [ ]:
sequenceClassifier_loaded = RoBertaForSequenceClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
sequenceClassifier_loaded.getClasses()

['disgust',
 'optimism',
 'embarrassment',
 'amusement',
 'realization',
 'surprise',
 'grief',
 'caring',
 'disapproval',
 'disappointment',
 'joy',
 'confusion',
 'excitement',
 'approval',
 'curiosity',
 'anger',
 'love',
 'admiration',
 'gratitude',
 'annoyance',
 'remorse',
 'nervousness',
 'neutral',
 'pride',
 'fear',
 'sadness',
 'desire',
 'relief']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["I love you!"], ['I feel lucky to be here.']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "class.result").show()

That's it! You can now go wild and use hundreds of `RoBertaForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
